## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius or on a unix system.

In [1]:
import os
from pathlib import Path
import subprocess
from PyStemmusScope import forcing_io
from PyStemmusScope import iostreamer

#### Path to STEMMUS_SCOPE

STEMMUS_SCOPE executable file is located in the **private** repository on GitHub https://github.com/EcoExtreML/STEMMUS_SCOPE. You need to clone the repository locally and specify the path to it in the cell below. Make sure you have right access to the repository. 

#### Update/set config files

In [2]:
# the user must provide the correct path
# path to config file
path_to_config_file = "path_to_config_file"
# Instantiate working directories handler from PyStemmusScope
config_template = iostreamer.read_config(path_to_config_file)

In [3]:
config_template

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '17520',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_2022-05-

In [4]:
# edit config
config_template["NumberOfTimeSteps"] = "5"
config_template

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '5',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_2022-05-11-1

#### Create input directories, prepare input files 

In [5]:
# specify the forcing filenames
# forcing_filenames_list is by default none, if full_run is true, then all stations with forcing
# listed in the ForcingPath will be used.
forcing_filenames_list = ["ZA-Kru_2000-2002_FLUXNET2015_Met.nc"]

full_run = False
if full_run:
    forcing_filenames_list = [file.name for file in Path(config_template["ForcingPath"]).iterdir()]

# empty dict to store path to config file and work directory for each station
config_path_dict = {}
input_dir_dict = {}
for nc_file in forcing_filenames_list:
    input_dir, output_dir, config_path = iostreamer.create_io_dir(nc_file, config_template)
    config_path_dict[nc_file] = config_path
    input_dir_dict[nc_file] = input_dir
    
    config_run = iostreamer.read_config(config_path)

    forcing_io.prepare_forcing(input_dir, Path(config_run["ForcingPath"]) / nc_file, config_run)
    
    print(input_dir, output_dir, config_path)

/scratch-shared/ecoextreml/stemmus_scope/input/ZA-Kru_20220621_1334 /scratch-shared/ecoextreml/stemmus_scope/output/ZA-Kru_20220621_1334 /scratch-shared/ecoextreml/stemmus_scope/input/ZA-Kru_20220621_1334/ZA-Kru_20220621_1334_config.txt


#### Run the model

If you run the model locally, you need to first intsall MATLAB Runtime, see instructions in the [readme](https://github.com/EcoExtreML/processing#readme). Then set `LD_LIBRARY_PATH`. To do this, uncomment the cell below and run it:

In [ ]:
# # Set LD_LIBRARY_PATH
# matlab_path = !whereis MATLAB
# matlab_path = matlab_path.s.split(": ")[1]
# os.environ['LD_LIBRARY_PATH'] = (
#     f"{matlab_path}/MATLAB_Runtime/v910/runtime/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/bin/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/sys/os/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/extern/bin/glnxa64:"
#     f"{matlab_path}/MATLAB_Runtime/v910/sys/opengl/lib/glnxa64")
# print(os.environ['LD_LIBRARY_PATH'])

In [6]:
# generate a text file
path_to_model = "path_to_STEMMUS_SCOPE_repository"
for ncfile in forcing_filenames_list:
    path_to_config = config_path_dict[ncfile]
    # set matlab log dir to slurm, otherwise java.log files are created in user home dir
    os.environ['MATLAB_LOG_DIR'] = str(input_dir_dict[ncfile])
    # run the model
    print(f"Run STEMMUS-SCOPE with the forcing: {ncfile}")
    result = subprocess.run([f"exe/STEMMUS_SCOPE {path_to_config}"], cwd = Path(path_to_model), shell=True)
    result.check_returncode()

Run STEMMUS-SCOPE with the forcing: ZA-Kru_2000-2002_FLUXNET2015_Met.nc
Opening log file:  /scratch-shared/ecoextreml/stemmus_scope/input/ZA-Kru_20220621_1334/java.log.20020
Reading config from /scratch-shared/ecoextreml/stemmus_scope/input/ZA-Kru_20220621_1334/ZA-Kru_20220621_1334_config.txt

 The calculations start now 


#### Create output directories, prepare output files

In [7]:
# convert csv files to nc files
path_to_generate_netcdf_script = "path_to_generate_netcdf_script"
path_to_Variables_will_be_in_NetCDF_file = "path_to_Variables_will_be_in_NetCDF_file"
for ncfile in forcing_filenames_list:
    path_to_config = config_path_dict[ncfile]
    result = subprocess.run(["python", path_to_generate_netcdf_script,
         "--config_file", path_to_config, "--variable_file",
         path_to_Variables_will_be_in_NetCDF_file])
    result.check_returncode()

/scratch-shared/ecoextreml/stemmus_scope/output/ZA-Kru_20220621_1334/ECdata.csv
Reading variable metadata from /home/yangl/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/output/ZA-Kru_20220621_1334/ZA-Kru_20220621_1334_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading data from file: fluxes.csv
Reading data from file: surftemp.csv
Reading data from file: Sim_Temp.csv
Reading data from file: Sim_Theta.csv
Reading data from file: aerodyn.csv
Reading data from file: ECdata.csv
Done writing /scratch-shared/ecoextreml/stemmus_scope/output/ZA-Kru_20220621_1334/ZA-Kru_20220621_1334_STEMMUS_SCOPE.nc
